# Performance Experiments Using TPC-H

## Imports

In [6]:
import snowflake.connector
import pandas as pd
import time
import os 
import plotly.express as px

## Setting up the connection

**Important: To run this, you must use your own credentials for ``“user”``, ``“password”``, and ``“account”``.**

1. **Set Up Your Password:**

    Create a ``.env`` file in the root directory and specify your password as follows:

    ``PASSWORD=YOUR_PASSWORD_HERE``
    
    For example: 
    
    ``PASSWORD=snowflake123``

2.	**Configure User and Account:**

    Update the **``“user”``** and **``“account”``** fields in the configuration section two cells below. Typically, the **``“account”``** I belive should be the same.

In [7]:
PASSWORD = os.getenv("PASSWORD")

In [8]:
conn = snowflake.connector.connect(
    user='COYOTE',             
    password=PASSWORD,    
    account='sfedu02-gyb58550',
    schema='TPCH_SF1',    
    warehouse='COYOTE_WH_XS',     
    database='SNOWFLAKE_SAMPLE_DATA'
)

## Warehouses and caching

In [5]:
warehouse_sizes = ['XS', 'S', 'M', 'L']
warehouse_real_sizes = ['XSMALL', 'SMALL', 'MEDIUM', 'LARGE']
schemas = ['TPCH_SF1', 'TPCH_SF10', 'TPCH_SF100', 'TPCH_SF1000']
warehouse_names = [f"COYOTE_WH_{size}" for size in warehouse_sizes]

for warehouse_name, size in zip(warehouse_names, warehouse_real_sizes):
    with conn.cursor() as cursor:
        cursor.execute(f"""
        CREATE OR REPLACE WAREHOUSE {warehouse_name}
        WAREHOUSE_SIZE = '{size}',
        AUTO_SUSPEND = 10;
        """)
        print(f"Created or replaced warehouse: {warehouse_name}")

Created or replaced warehouse: COYOTE_WH_XS
Created or replaced warehouse: COYOTE_WH_S
Created or replaced warehouse: COYOTE_WH_M
Created or replaced warehouse: COYOTE_WH_L


In [6]:
with conn.cursor() as cursor:
    cursor.execute("ALTER SESSION SET USE_CACHED_RESULT = FALSE;")
print("Caching has been turned off for the session.")

Caching has been turned off for the session.


## Queries (taken from  https://docs.snowflake.com/en/user-guide/sample-data-tpch -> https://www.tpc.org/tpc_documents_current_versions/pdf/tpc-h_v2.17.1.pdf)

In [7]:
queries = [
    {
        'name': 'Query1',
        'sql': """
        SELECT
            l_returnflag,
            l_linestatus,
            sum(l_quantity) as sum_qty,
            sum(l_extendedprice) as sum_base_price,
            sum(l_extendedprice * (1-l_discount)) as sum_disc_price,
            sum(l_extendedprice * (1-l_discount) * (1+l_tax)) as sum_charge,
            avg(l_quantity) as avg_qty,
            avg(l_extendedprice) as avg_price,
            avg(l_discount) as avg_disc,
            count(*) as count_order
        FROM
            lineitem
        WHERE
            l_shipdate <= dateadd(day, -90, to_date('1998-12-01'))
        GROUP BY
            l_returnflag,
            l_linestatus
        ORDER BY
            l_returnflag,
            l_linestatus;
        """
    },
    {
        'name': 'Query5',
        'sql': """
        SELECT
            n_name,
            SUM(l_extendedprice * (1 - l_discount)) AS revenue
        FROM
            customer,
            orders,
            lineitem,
            supplier,
            nation,
            region
        WHERE
            c_custkey = o_custkey
            AND l_orderkey = o_orderkey
            AND l_suppkey = s_suppkey
            AND c_nationkey = s_nationkey
            AND s_nationkey = n_nationkey
            AND n_regionkey = r_regionkey
            AND r_name = 'ASIA'
            AND o_orderdate >= DATE '1994-01-01'
            AND o_orderdate < DATEADD(year, 1, '1994-01-01')
        GROUP BY
            n_name
        ORDER BY
            revenue DESC;
        """
    },
    {
        'name': 'Query18',
        'sql': """
        SELECT
            c_name,
            c_custkey,
            o_orderkey,
            o_orderdate,
            o_totalprice,
            SUM(l_quantity)
        FROM
            customer,
            orders,
            lineitem
        WHERE
            o_orderkey IN (
                SELECT
                    l_orderkey
                FROM
                    lineitem
                GROUP BY
                    l_orderkey
                HAVING
                    SUM(l_quantity) > 313
            )
            AND c_custkey = o_custkey
            AND o_orderkey = l_orderkey
        GROUP BY
            c_name,
            c_custkey,
            o_orderkey,
            o_orderdate,
            o_totalprice
        ORDER BY
            o_totalprice DESC,
            o_orderdate;
        """
    }
]

## Experiments

Takes around ~1 hour to run all the combinations. 

I am running each query 3 times.

In [7]:
results = []
username = 'COYOTE'  
# run_number = 1

for warehouse_size in warehouse_sizes:
    with conn.cursor() as cursor:
        cursor.execute(f"USE WAREHOUSE COYOTE_WH_{warehouse_size}")
    print(f"Switched to warehouse: {warehouse_size}")
    
    for schema_name in schemas:
        with conn.cursor() as cursor:
            cursor.execute(f"USE SCHEMA {schema_name}")
        print(f"Switched to schema: {schema_name}")
        
        for query in queries:
            query_name = query['name']
            query_sql = query['sql']
            
            for run_number in range(1, 4):
                with conn.cursor() as cursor:
                    try:
                        cursor.execute(query_sql)
                        query_id = cursor.sfqid 
                        print(f"Executed {query_name}, Run {run_number}, Query ID: {query_id}")
                        
                        results.append({
                            'query_id': query_id,
                            'schema_name': schema_name,
                            'warehouse_size': warehouse_size,
                            'query_name': query_name,
                            'run_number': run_number
                        })
                        
                    except Exception as e:
                        print(f"Error executing {query_name}: {e}")
                    
                    # time.sleep(2)

Switched to warehouse: XS
Switched to schema: TPCH_SF1
Executed Query1, Run 1, Query ID: 01b88fde-0004-38e7-0000-137f00577f9a
Executed Query1, Run 2, Query ID: 01b88fde-0004-3924-0000-137f0057860a
Executed Query1, Run 3, Query ID: 01b88fdf-0004-38e7-0000-137f00577f9e
Executed Query5, Run 1, Query ID: 01b88fdf-0004-3924-0000-137f0057860e
Executed Query5, Run 2, Query ID: 01b88fdf-0004-38e7-0000-137f00577fa2
Executed Query5, Run 3, Query ID: 01b88fdf-0004-3924-0000-137f00578612
Executed Query18, Run 1, Query ID: 01b88fdf-0004-38e7-0000-137f00577fa6
Executed Query18, Run 2, Query ID: 01b88fdf-0004-3924-0000-137f00578616
Executed Query18, Run 3, Query ID: 01b88fdf-0004-38e7-0000-137f00577faa
Switched to schema: TPCH_SF10
Executed Query1, Run 1, Query ID: 01b88fdf-0004-38e7-0000-137f00577fae
Executed Query1, Run 2, Query ID: 01b88fdf-0004-3924-0000-137f0057861e
Executed Query1, Run 3, Query ID: 01b88fdf-0004-38e7-0000-137f00577fb2
Executed Query5, Run 1, Query ID: 01b88fdf-0004-3924-0000-13

## Retrieving and saving the results

In [8]:
query_ids = [result['query_id'] for result in results]
query_ids_str = ','.join([f"'{qid}'" for qid in query_ids])

query_history_sql = f"""
SELECT
    query_id,
    schema_name,
    warehouse_name AS warehouse_size,
    total_elapsed_time/1000 AS time_elapsed_in_seconds
FROM
    table(information_schema.query_history(RESULT_LIMIT => 1000))
WHERE
    user_name = '{username.upper()}'
    AND execution_status = 'SUCCESS'
    AND query_id IN ({query_ids_str})
    ORDER BY start_time
"""

with conn.cursor() as cursor:
    cursor.execute(query_history_sql)
    df_history = pd.DataFrame.from_records(iter(cursor), columns=[x[0].lower() for x in cursor.description])

df_history.columns = df_history.columns.str.lower()

df_history['query_id'] = df_history['query_id'].str.upper()
results_df = pd.DataFrame(results)
results_df['query_id'] = results_df['query_id'].str.upper()

final_df = pd.merge(results_df, df_history, on='query_id')

for col in ['schema_name', 'warehouse_size']:
    if f"{col}_x" in final_df.columns:
        final_df.rename(columns={f"{col}_x": col}, inplace=True)
    elif f"{col}_results" in final_df.columns:
        final_df.rename(columns={f"{col}_results": col}, inplace=True)
    elif f"{col}_y" in final_df.columns:
        final_df.rename(columns={f"{col}_y": col}, inplace=True)
    elif f"{col}_history" in final_df.columns:
        final_df.rename(columns={f"{col}_history": col}, inplace=True)

final_df = final_df[['query_id', 'query_name', 'schema_name', 'warehouse_size', 'run_number', 
                     'time_elapsed_in_seconds']]

final_df.to_csv('../tpch_benchmark/query_execution_times.csv', index=False)

In [9]:
grouped_df = final_df.groupby(['query_name', 'schema_name', 'warehouse_size'])

average_times_df = grouped_df['time_elapsed_in_seconds'].mean().reset_index()

average_times_df.rename(columns={'time_elapsed_in_seconds': 'average_time_seconds'}, inplace=True)

average_times_df.sort_values(by='average_time_seconds', inplace=True)
average_times_df.to_csv('../tpch_benchmark/average_query_execution_times.csv', index=False)

In [10]:
average_times_df = pd.read_csv('../tpch_benchmark/average_query_execution_times.csv')
average_times_df

,query_name,schema_name,warehouse_size,average_time_seconds
0,Query18,TPCH_SF1,L,0.267667
1,Query18,TPCH_SF1,M,0.295667
2,Query18,TPCH_SF1,S,0.309667
3,Query1,TPCH_SF1,L,0.364333
4,Query18,TPCH_SF1,XS,0.367333
5,Query1,TPCH_SF1,XS,0.390333
6,Query1,TPCH_SF10,L,0.402667
7,Query5,TPCH_SF1,L,0.448333
8,Query1,TPCH_SF1,S,0.456000
9,Query5,TPCH_SF1,XS,0.479000


## Plotting the Results

In [11]:
color_map = {
    'XS': 'red',  
    'S': 'blue',
    'M': 'green',
    'L': 'purple'
}

query_order = ['Query1', 'Query5', 'Query18'] 

fig = px.line(average_times_df, 
              x='schema_name', 
              y='average_time_seconds', 
              color='warehouse_size', 
              symbol="warehouse_size",
              color_discrete_map=color_map, 
              category_orders={
                  "warehouse_size": warehouse_sizes, 
                  "query_name": query_order  
              },
              facet_col="query_name", 
              labels={
                  "schema_name": "Schema",
                  "average_time_seconds": "Average Time (seconds)"
              })

fig.update_layout(
    legend_title="Warehouse Size",
    xaxis_title="Schema"
)

fig.update_yaxes(type='log', 
                 tickmode='array',
                 tickvals=[0.1, 1, 10, 100, 1000],
                 autorange=False,
                 range=[-1, 3] 
                )

fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))

fig.show()